In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑
import sys
from pathlib import Path
# new_version_path = Path(r'D:\vnpy-2.0.8')
new_version_path = Path(r'E:\vnpy\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

import vnpy
print(vnpy.__version__)

In [ ]:
import pandas as pd
from datetime import datetime
from vnpy.app.cta_strategy.backtesting import BacktestingEngine

from utility import comodity_to_vt_symbol, get_output_path
from trade_match import calculate_trades_result, generate_trade_df, exhaust_trade_result
from turtle_b_strategy import TurtleBStrategy

future_basic_data = pd.read_csv('future_basic_data.csv', index_col=0)

In [ ]:
commodity = 'jd'
data_mode = 'main'
interval = '1h'
strategy_name = 'turtle'
strategy_params = {}
capital = 100000
start = datetime(2015, 1, 1)
end = datetime(2019, 12, 1)

vt_symbol = comodity_to_vt_symbol(commodity, 'main')
size = future_basic_data.loc[commodity]['size']
pricetick = future_basic_data.loc[commodity]['pricetick']

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol=vt_symbol,
    interval=interval,
    start=start,
    end=end,
    rate=0.0001,
    slippage=pricetick,
    size=size,
    pricetick=pricetick,
    capital=capital
)
engine.add_strategy(TurtleBStrategy, strategy_params)

In [ ]:
# run backtest
engine.load_data()
engine.run_backtesting()


In [ ]:
pnl_df = engine.calculate_result()
pnl_df.to_csv(get_output_path('pnl_day_by_day.csv'))

engine.calculate_statistics()

In [ ]:
# save trade
trades = engine.trades

df = calculate_trades_result(trades)
df
# df.head()

In [ ]:
df2 = generate_trade_df(trades, size, 0.0001, pricetick, capital)
df2

In [ ]:
trades = engine.trades
exhaust_trade_result(trades, size, 0.0001, pricetick, capital)